Running Your Queries In Spark
---------------------------

You need to take the data from Twitter and perform your analysis based on the question you chose.

In our hashtag counting example, we do the following:
1. Explode the tweet into its individual words.
2. Filter out hashtags, then each tag gets assigned the count 1 with the `map` method.
3. Append the current hashtags to the main dataframe and update the counts appropriately.
4. Query the temporary table to return the current top 10 hashtags trending, and show to the console.

You can extend this into a web dashboard, or plots inside this notebook if you choose.

----------------------------------------------------------------------------------------------------------------------

#### Import the relevant modules

In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext, SparkSession
import sys
import requests

#### Set some constants, initialize Spark, and then open the socket with the remote host.

In [2]:
TCP_REMOTE_HOST = "data_server"
TCP_PORT = 9009

# create spark configuration
conf = SparkConf()
conf.setAppName("TwitterStreamApp")

# create spark context with the above configuration
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")

# create the Streaming Context from the above spark context with interval size 2 seconds
ssc = StreamingContext(sc, 2)

# setting a checkpoint to allow RDD recovery
ssc.checkpoint("checkpoint_TwitterApp")

# read data from port 9009
dataStream = ssc.socketTextStream(TCP_REMOTE_HOST, TCP_PORT)

#### We now create some helper functions to allow Spark to maintain our running count.

In [3]:
def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

def getSparkSessionInstance(sparkConf):
    if ("sparkSessionSingletonInstance" not in globals()):
        globals()["sparkSessionSingletonInstance"] = SparkSession \
            .builder \
            .config(conf=sparkConf) \
            .getOrCreate()
    return globals()["sparkSessionSingletonInstance"]

def process_rdd(time, rdd):
    print(f"----------- {str(time)} -----------")
    try:
        # Get spark sql singleton context from the current context
        spark = getSparkSessionInstance(rdd.context.getConf())
        
        # convert the RDD to Row RDD
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
        
        # create a DF from the Row RDD
        hashtags_df = spark.createDataFrame(row_rdd)
        
        # Register the dataframe as table
        hashtags_df.registerTempTable("hashtags")
        #hashtags_df.createOrReplaceTempView("hashtags")
        
        # get the top 10 hashtags from the table using SQL and print them
        hashtag_counts_df = spark.sql(
            "select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10")
        hashtag_counts_df.show()

    except Exception as e:
        e = sys.exc_info()[1]
        print(f"Error: {e}")

#### Finally, we assign our primary workflow that will utilize the above functions.

After that's complete, we begin the streaming with `ssc.start()`. The query stays open until we terminate it (`ssc.awaitTermination()`).

We can end the streaming by going to the `data_server` container and typing `ctrl-C`. Recall that you can exit the container shell with the `Ctrl-P, Ctrl-Q` sequence.

If you are using tweets from a geofenced area (as we are here) - you should let it run for a while in order to build up enough data to be useful.

In [5]:
# split each tweet into words
words = dataStream.flatMap(lambda line: line.split(" "))

# filter the words to get only hashtags, then map each hashtag to be a pair of (hashtag,1)
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x, 1))

# adding the count of each hashtag to its last count
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)

# do processing for each RDD generated in each interval
tags_totals.foreachRDD(process_rdd)

# start the streaming computation
ssc.start()

# wait for the streaming to finish
ssc.awaitTermination()

+--------+-------------+
| hashtag|hashtag_count|
+--------+-------------+
|#LHHNYNo|            1|
+--------+-------------+



Py4JJavaError: An error occurred while calling None.org.apache.spark.streaming.api.python.PythonTransformedDStream.
: java.lang.IllegalStateException: Adding new inputs, transformations, and output operations after starting a context is not supported
	at org.apache.spark.streaming.dstream.DStream.validateAtInit(DStream.scala:224)
	at org.apache.spark.streaming.dstream.DStream.<init>(DStream.scala:66)
	at org.apache.spark.streaming.api.python.PythonDStream.<init>(PythonDStream.scala:224)
	at org.apache.spark.streaming.api.python.PythonTransformedDStream.<init>(PythonDStream.scala:241)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)


----------- 2018-12-18 06:58:52 -----------
+--------+-------------+
| hashtag|hashtag_count|
+--------+-------------+
|#LHHNYNo|            1|
+--------+-------------+

----------- 2018-12-18 06:58:54 -----------
+--------+-------------+
| hashtag|hashtag_count|
+--------+-------------+
|#LHHNYNo|            1|
+--------+-------------+

----------- 2018-12-18 06:58:56 -----------
+--------+-------------+
| hashtag|hashtag_count|
+--------+-------------+
|#LHHNYNo|            1|
+--------+-------------+

----------- 2018-12-18 06:58:58 -----------
+--------+-------------+
| hashtag|hashtag_count|
+--------+-------------+
|#LHHNYNo|            1|
+--------+-------------+

----------- 2018-12-18 06:59:00 -----------
+--------+-------------+
| hashtag|hashtag_count|
+--------+-------------+
|#LHHNYNo|            1|
+--------+-------------+

----------- 2018-12-18 06:59:02 -----------
+--------+-------------+
| hashtag|hashtag_count|
+--------+-------------+
|#LHHNYNo|            1|
+-----

----------- 2018-12-18 07:00:02 -----------
+-------------+-------------+
|      hashtag|hashtag_count|
+-------------+-------------+
|  #tarantulas|            1|
|     #LHHNYNo|            1|
|#theexplosion|            1|
|    #brooklyn|            1|
+-------------+-------------+

----------- 2018-12-18 07:00:04 -----------
+-------------+-------------+
|      hashtag|hashtag_count|
+-------------+-------------+
|  #tarantulas|            1|
|     #LHHNYNo|            1|
|#theexplosion|            1|
|    #brooklyn|            1|
+-------------+-------------+

----------- 2018-12-18 07:00:06 -----------
+-------------+-------------+
|      hashtag|hashtag_count|
+-------------+-------------+
|  #tarantulas|            1|
|     #LHHNYNo|            1|
|#theexplosion|            1|
|    #brooklyn|            1|
+-------------+-------------+

----------- 2018-12-18 07:00:08 -----------
+-------------+-------------+
|      hashtag|hashtag_count|
+-------------+-------------+
|  #tarantu